#### README

I wrote this notebook to more easily load my covid data into MySQL Workbench. I found that the given Import tool did not work either quickly or completely, so I had to use the command line. 

This notebook serves two purposes:
1. to write potentially-very-long CREATE TABLE statements automatically
2. to clean the data appropriately for the LOAD DATA command


The final 2 cells write the CREATE TABLE statement and export the cleaned data to a new CSV

In [18]:
import pandas as pd
import numpy as np

df = pd.read_csv('covid_deaths.csv')
df.head()

,iso_code,continent,location,date,population,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,...,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million
0,AFG,Asia,Afghanistan,2/24/20,38928341.0,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2/25/20,38928341.0,1.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2/26/20,38928341.0,1.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2/27/20,38928341.0,1.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2/28/20,38928341.0,1.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# Replace nulls with '\N' so LOAD DATA statement can properly parse the file

df = df.replace(np.NaN,'\\N')
df.head()

,iso_code,continent,location,date,population,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,...,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million
0,AFG,Asia,Afghanistan,2/24/20,38928341.0,1.0,1.0,\N,\N,\N,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1,AFG,Asia,Afghanistan,2/25/20,38928341.0,1.0,0.0,\N,\N,\N,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
2,AFG,Asia,Afghanistan,2/26/20,38928341.0,1.0,0.0,\N,\N,\N,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
3,AFG,Asia,Afghanistan,2/27/20,38928341.0,1.0,0.0,\N,\N,\N,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
4,AFG,Asia,Afghanistan,2/28/20,38928341.0,1.0,0.0,\N,\N,\N,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N


In [8]:
for col in df.columns:
    print(df[col].dtype)

int64
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object


In [10]:
list(df.columns)

['UniqueID ',
 'ParcelID',
 'LandUse',
 'PropertyAddress',
 'SaleDate',
 'SalePrice',
 'LegalReference',
 'SoldAsVacant',
 'OwnerName',
 'OwnerAddress',
 'Acreage',
 'TaxDistrict',
 'LandValue',
 'BuildingValue',
 'TotalValue',
 'YearBuilt',
 'Bedrooms',
 'FullBath',
 'HalfBath']

In [11]:
def write_sql_script(cols, table_name):
    string = 'CREATE TABLE ' + table_name + ' ('
    for col in cols:
        string += col + ' '
        if df[col].dtype == 'int64':
            string += 'INT, '
        elif df[col].dtype == 'object':
            string += 'TEXT, '
        elif df[col].dtype == 'float64':
            string += 'DOUBLE, '
        else:
            print('Panic!')
    string = string.rstrip(', ')
    string += ')'
    print(string)

write_sql_script(df.columns, 'nashvillehousing')        
    

CREATE TABLE nashvillehousing (UniqueID  INT, ParcelID TEXT, LandUse TEXT, PropertyAddress TEXT, SaleDate TEXT, SalePrice TEXT, LegalReference TEXT, SoldAsVacant TEXT, OwnerName TEXT, OwnerAddress TEXT, Acreage TEXT, TaxDistrict TEXT, LandValue TEXT, BuildingValue TEXT, TotalValue TEXT, YearBuilt TEXT, Bedrooms TEXT, FullBath TEXT, HalfBath TEXT)


In [12]:
df.to_csv('coviddeaths1.csv', encoding='utf-8', index=False)